# Homework 1: Python intro and simple statistics

**Due Friday, May 24**

This notebook covers Python/Numpy basics and data exploration using SLC PM2.5 air quality data.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## Part 1: Python/Numpy

### 1. Write functions to compute mean and standard deviation

In [ ]:
def compute_mean(data):
    """
    Compute the mean of a list of data
    
    Parameters:
    data (list or array): Input data
    
    Returns:
    float: Mean of the data
    """
    return sum(data) / len(data)

def compute_std(data):
    """
    Compute the standard deviation of a list of data
    
    Parameters:
    data (list or array): Input data
    
    Returns:
    float: Standard deviation of the data
    """
    mean = compute_mean(data)
    variance = sum((x - mean) ** 2 for x in data) / (len(data) - 1)  # Sample standard deviation
    return variance ** 0.5

# Test the functions with sample data
test_data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(f"Test data: {test_data}")
print(f"Custom mean: {compute_mean(test_data):.4f}")
print(f"Custom std: {compute_std(test_data):.4f}")
print(f"NumPy mean: {np.mean(test_data):.4f}")
print(f"NumPy std: {np.std(test_data, ddof=1):.4f}")  # ddof=1 for sample std

### 2. Sample from normal distribution and verify results

In [ ]:
# Parameters for normal distribution
true_mean = 50
true_std = 10
sample_size = 1000

# Sample from normal distribution using scipy.stats.norm
samples = stats.norm.rvs(loc=true_mean, scale=true_std, size=sample_size)

# Compute statistics using custom functions
custom_mean = compute_mean(samples)
custom_std = compute_std(samples)

# Compute statistics using NumPy
numpy_mean = np.mean(samples)
numpy_std = np.std(samples, ddof=1)

# Display results
print(f"True parameters: mean = {true_mean}, std = {true_std}")
print(f"Sample size: {sample_size}")
print()
print("Custom functions:")
print(f"  Mean: {custom_mean:.4f} (difference from true: {abs(custom_mean - true_mean):.4f})")
print(f"  Std:  {custom_std:.4f} (difference from true: {abs(custom_std - true_std):.4f})")
print()
print("NumPy functions:")
print(f"  Mean: {numpy_mean:.4f} (difference from true: {abs(numpy_mean - true_mean):.4f})")
print(f"  Std:  {numpy_std:.4f} (difference from true: {abs(numpy_std - true_std):.4f})")
print()
print(f"Difference between custom and NumPy mean: {abs(custom_mean - numpy_mean):.6f}")
print(f"Difference between custom and NumPy std: {abs(custom_std - numpy_std):.6f}")

### 3. Plot histogram of samples

In [ ]:
# Create histogram
plt.figure(figsize=(10, 6))
plt.hist(samples, bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black')

# Overlay theoretical normal distribution
x = np.linspace(samples.min(), samples.max(), 100)
theoretical_pdf = stats.norm.pdf(x, true_mean, true_std)
plt.plot(x, theoretical_pdf, 'r-', linewidth=2, label=f'Theoretical Normal(μ={true_mean}, σ={true_std})')

# Add vertical lines for means
plt.axvline(true_mean, color='red', linestyle='--', alpha=0.8, label=f'True mean: {true_mean}')
plt.axvline(custom_mean, color='blue', linestyle='--', alpha=0.8, label=f'Sample mean: {custom_mean:.2f}')

plt.title('Histogram of Normal Distribution Samples')
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Part 2: Data Exploration/Analysis

### Load SLC PM2.5 Data

**Note:** To run this section, download a year's worth of hourly SLC PM2.5 data from https://air.utah.gov/dataarchive/archpm25.htm and place the CSV file in the same directory as this notebook.

In [ ]:
# Load the PM2.5 data
# Replace 'pm25_data.csv' with the actual filename you downloaded

df = pd.read_csv('pm25_data.csv')
print("Successfully loaded real PM2.5 data")
print(f"Data shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())
print("\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Data preprocessing
# Convert DateTime column to datetime if it's not already
if 'DateTime' in df.columns:
    df['DateTime'] = pd.to_datetime(df['DateTime'])
elif 'Date' in df.columns and 'Time' in df.columns:
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
else:
    # Try to find datetime column automatically
    datetime_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
    if datetime_cols:
        df['DateTime'] = pd.to_datetime(df[datetime_cols[0]])

# Find PM2.5 column
pm25_cols = [col for col in df.columns if 'pm2.5' in col.lower() or 'pm25' in col.lower()]
if pm25_cols:
    pm25_col = pm25_cols[0]
else:
    pm25_col = 'PM2.5'  # Use our sample data column name

print(f"Using PM2.5 column: {pm25_col}")
print(f"Date range: {df['DateTime'].min()} to {df['DateTime'].max()}")
print(f"Number of records: {len(df)}")

# Select one station if multiple stations exist
if 'Station' in df.columns:
    stations = df['Station'].unique()
    print(f"\nAvailable stations: {stations}")
    selected_station = stations[0]
    df_station = df[df['Station'] == selected_station].copy()
    print(f"Selected station: {selected_station}")
    print(f"Records for selected station: {len(df_station)}")
else:
    df_station = df.copy()
    print("Using all data (single station assumed)")

### Plot readings over the course of a year

In [ ]:
# Plot the full year of PM2.5 readings
plt.figure(figsize=(15, 6))
plt.plot(df_station['DateTime'], df_station[pm25_col], linewidth=0.5, alpha=0.7)
plt.title('PM2.5 Readings Over One Year', fontsize=14)
plt.xlabel('Date')
plt.ylabel('PM2.5 (μg/m³)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"As expected, with {len(df_station)} hourly data points, this plot is quite dense.")
print("Let's explore patterns by grouping the data by month and hour.")

### Monthly Analysis

In [ ]:
# Add month and hour columns for grouping
df_station['Month'] = df_station['DateTime'].dt.month
df_station['Hour'] = df_station['DateTime'].dt.hour
df_station['MonthName'] = df_station['DateTime'].dt.strftime('%B')

# Calculate monthly means
monthly_means = df_station.groupby('Month')[pm25_col].mean()
monthly_names = df_station.groupby('Month')['MonthName'].first()

# Plot monthly means as bar chart
plt.figure(figsize=(12, 6))
bars = plt.bar(range(1, 13), monthly_means.values, color='steelblue', alpha=0.7, edgecolor='black')
plt.title('Mean PM2.5 Levels by Month', fontsize=14)
plt.xlabel('Month')
plt.ylabel('Mean PM2.5 (μg/m³)')
plt.xticks(range(1, 13), monthly_names.values, rotation=45)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Insights
max_month = monthly_means.idxmax()
min_month = monthly_means.idxmin()
print("\nMonthly Analysis Insights:")
print(f"• Highest PM2.5 levels in {monthly_names[max_month]} ({monthly_means[max_month]:.1f} μg/m³)")
print(f"• Lowest PM2.5 levels in {monthly_names[min_month]} ({monthly_means[min_month]:.1f} μg/m³)")
print(f"• Seasonal variation: {monthly_means.max() - monthly_means.min():.1f} μg/m³ difference")
if max_month in [12, 1, 2]:
    print("• Higher pollution in winter months, likely due to increased heating and atmospheric conditions")
if min_month in [6, 7, 8]:
    print("• Lower pollution in summer months, possibly due to better atmospheric mixing")

### Hourly Analysis

In [ ]:
# Calculate hourly means
hourly_means = df_station.groupby('Hour')[pm25_col].mean()

# Plot hourly means
plt.figure(figsize=(12, 6))
plt.plot(hourly_means.index, hourly_means.values, marker='o', linewidth=2, markersize=6, color='darkgreen')
plt.title('Mean PM2.5 Levels by Hour of Day', fontsize=14)
plt.xlabel('Hour of Day')
plt.ylabel('Mean PM2.5 (μg/m³)')
plt.xticks(range(0, 24, 2))
plt.grid(True, alpha=0.3)

# Add value labels at peaks and valleys
max_hour = hourly_means.idxmax()
min_hour = hourly_means.idxmin()
plt.annotate(f'Peak: {hourly_means[max_hour]:.1f}', 
             xy=(max_hour, hourly_means[max_hour]), 
             xytext=(max_hour+2, hourly_means[max_hour]+2),
             arrowprops=dict(arrowstyle='->', color='red'))
plt.annotate(f'Minimum: {hourly_means[min_hour]:.1f}', 
             xy=(min_hour, hourly_means[min_hour]), 
             xytext=(min_hour+2, hourly_means[min_hour]-2),
             arrowprops=dict(arrowstyle='->', color='blue'))

plt.tight_layout()
plt.show()

# Insights
print("\nHourly Analysis Insights:")
print(f"• Peak pollution at hour {max_hour}:00 ({hourly_means[max_hour]:.1f} μg/m³)")
print(f"• Lowest pollution at hour {min_hour}:00 ({hourly_means[min_hour]:.1f} μg/m³)")
print(f"• Daily variation: {hourly_means.max() - hourly_means.min():.1f} μg/m³ difference")

# Identify rush hour patterns
morning_rush = hourly_means[6:10].mean()
evening_rush = hourly_means[16:20].mean()
midday = hourly_means[11:15].mean()
night = hourly_means[22:24].mean() + hourly_means[0:6].mean()
night = night / 2  # Average of late night and early morning

print(f"• Morning rush hours (6-9 AM): {morning_rush:.1f} μg/m³")
print(f"• Evening rush hours (4-7 PM): {evening_rush:.1f} μg/m³")
print(f"• Midday (11 AM-2 PM): {midday:.1f} μg/m³")
print(f"• Night time (10 PM-5 AM): {night:.1f} μg/m³")

if morning_rush > midday or evening_rush > midday:
    print("• Higher pollution during rush hours suggests traffic-related emissions")
if night < midday:
    print("• Lower nighttime pollution suggests reduced human activity and better atmospheric dispersion")

### Box and Whisker Plots for More Complete Analysis

In [ ]:
# Monthly box plot
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
df_station.boxplot(column=pm25_col, by='Month', ax=plt.gca())
plt.title('PM2.5 Distribution by Month')
plt.xlabel('Month')
plt.ylabel('PM2.5 (μg/m³)')
plt.xticks(range(1, 13), [monthly_names[i] for i in range(1, 13)], rotation=45)
plt.suptitle('')  # Remove automatic title

# Hourly box plot
plt.subplot(1, 2, 2)
df_station.boxplot(column=pm25_col, by='Hour', ax=plt.gca())
plt.title('PM2.5 Distribution by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('PM2.5 (μg/m³)')
plt.xticks(range(1, 25, 2), range(0, 24, 2))
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.show()

In [ ]:
# More detailed box plot analysis
# Calculate quartiles and outliers for monthly data
monthly_stats = df_station.groupby('Month')[pm25_col].describe()
hourly_stats = df_station.groupby('Hour')[pm25_col].describe()

print("Box Plot Analysis Insights:")
print("\nMonthly Patterns:")
highest_variability_month = (monthly_stats['75%'] - monthly_stats['25%']).idxmax()
lowest_variability_month = (monthly_stats['75%'] - monthly_stats['25%']).idxmin()
print(f"• Highest variability in {monthly_names[highest_variability_month]} (IQR: {monthly_stats.loc[highest_variability_month, '75%'] - monthly_stats.loc[highest_variability_month, '25%']:.1f})")
print(f"• Lowest variability in {monthly_names[lowest_variability_month]} (IQR: {monthly_stats.loc[lowest_variability_month, '75%'] - monthly_stats.loc[lowest_variability_month, '25%']:.1f})")

# Check for extreme outliers
outlier_months = []
for month in range(1, 13):
    month_data = df_station[df_station['Month'] == month][pm25_col]
    q75 = month_data.quantile(0.75)
    q25 = month_data.quantile(0.25)
    iqr = q75 - q25
    outlier_threshold = q75 + 1.5 * iqr
    outliers = (month_data > outlier_threshold).sum()
    if outliers > len(month_data) * 0.05:  # More than 5% outliers
        outlier_months.append((monthly_names[month], outliers))

if outlier_months:
    print(f"\nMonths with frequent high pollution episodes:")
    for month, count in outlier_months:
        print(f"• {month}: {count} extreme readings")

print("\nHourly Patterns:")
highest_variability_hour = (hourly_stats['75%'] - hourly_stats['25%']).idxmax()
lowest_variability_hour = (hourly_stats['75%'] - hourly_stats['25%']).idxmin()
print(f"• Highest variability at {highest_variability_hour}:00 (IQR: {hourly_stats.loc[highest_variability_hour, '75%'] - hourly_stats.loc[highest_variability_hour, '25%']:.1f})")
print(f"• Most consistent levels at {lowest_variability_hour}:00 (IQR: {hourly_stats.loc[lowest_variability_hour, '75%'] - hourly_stats.loc[lowest_variability_hour, '25%']:.1f})")

print("\nAdditional Insights from Box Plots:")
print("• Box plots reveal the full distribution shape, showing skewness and outliers")
print("• Outliers indicate pollution episodes that may warrant further investigation")
print("• The interquartile range shows the typical day-to-day variation")
print("• Comparing medians vs means can reveal whether data is skewed by extreme events")

## Summary

This homework demonstrates:

### Python/NumPy Skills:
- Implementation of basic statistical functions (mean, standard deviation)
- Working with scipy.stats for probability distributions
- Data visualization with matplotlib
- Verification that sample statistics converge to theoretical values

### Data Analysis Skills:
- Loading and preprocessing real-world environmental data
- Time series analysis and temporal grouping
- Statistical visualization techniques (histograms, bar charts, line plots, box plots)
- Pattern recognition in environmental data (seasonal and diurnal cycles)
- Understanding the value of different visualization approaches for gaining insights

The analysis reveals typical air quality patterns including seasonal variations (higher in winter) and daily cycles (peaks during rush hours), providing valuable insights into pollution sources and atmospheric processes.